# Build podcasts database

In [16]:
import json
import pickle
import requests
import pandas as pd
from copy import copy
from bs4 import BeautifulSoup

In [53]:
pd_dict = {
    "title": [],
    "aggregator": [],
    "category": [],
    "subcategory": [],
    "score": []
}

## Get Top Yandex

In [54]:
%%time

r = requests.get("https://music.yandex.ru/handlers/non-music.jsx?category=&postfix=&path=non-music&start=0&lang=ru&external-domain=music.yandex.ru&overembed=false")
yp = json.loads(r.text)
walk_yp = [(e["data"]["url"][19:].split("/")[1], e["data"]["title"]) for e in yp["blocks"][-1]["entities"]]

for url, title in walk_yp:
    r = requests.get("https://music.yandex.ru/handlers/non-music.jsx?category={}&postfix=&path=non-music&start=0&lang=ru&external-domain=music.yandex.ru&overembed=false".format(url))
    sub_yp = json.loads(r.text)
    for block in sub_yp["blocks"]:
        if "Все в категории" in block["title"]:
            continue
        pd_dict["aggregator"] += ["Top Yandex" for _ in range(len(block["entities"]))]
        pd_dict["category"] += [title for _ in range(len(block["entities"]))]
        pd_dict["subcategory"] += [block["title"] for _ in range(len(block["entities"]))]
        pd_dict["score"] += [None for _ in range(len(block["entities"]))]
        pd_dict["title"] += [e["data"]["title"] for e in block["entities"]]

CPU times: user 250 ms, sys: 78.1 ms, total: 328 ms
Wall time: 8.16 s


## Get Google

In [8]:
!curl 'https://podcasts.google.com/' \
  -H 'authority: podcasts.google.com' \
  -H 'cache-control: max-age=0' \
  -H 'sec-ch-ua: " Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
  -H 'x-client-data: CKe1yQEIk7bJAQiltskBCMS2yQEIqZ3KAQimlssBCNGaywEIoKDLAQjAoMsBCPbtywEI3fLLAQib88sBCMT2ywEIivjLAQiz+MsBGI6eywEYuvLLARiS9csB' \
  -H 'sec-fetch-site: same-origin' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-user: ?1' \
  -H 'sec-fetch-dest: document' \
  -H 'referer: https://www.google.com/' \
  -H 'accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'cookie: CONSENT=YES+RU.ru+20160717-08-0; HSID=Ag_qepdlEjkb4lMDF; SSID=AmeA16v13fBK8VFrM; APISID=pqh3sLwS5fxAXWSN/A_s3wWgVJMbBeLSd3; SAPISID=4RlXIOSaclTClibF/AOzvrsEhvTF7oohTq; __Secure-3PAPISID=4RlXIOSaclTClibF/AOzvrsEhvTF7oohTq; OGPC=19022519-1:19023808-1:19024399-1:; SID=-AcheRRCEKW2OKJBMtVDYPScRvScUnGNS9WgHekiE0NZ9RNFzICFeS69m00CebFp87j6gw.; __Secure-3PSID=-AcheRRCEKW2OKJBMtVDYPScRvScUnGNS9WgHekiE0NZ9RNFYK-LO35c2qmu2T6zo1F7ew.; SEARCH_SAMESITE=CgQI65IB; NID=217=oPnBSrJAtlgDebwmKQn3O-Xlnib9r2oJA9WGpg-Ygmq0Ye-TKRa_cDD42Zk-FgwlvpUdQPqgfm3OWKx3pTjGUJujMF0MLkw9dfwTPv9CYxd3R78GiZWzF2RPEUfGxOAb5tXbViQ6DBUn1s0ahscJJoUlohpOIyCAOOa_rSqpYn48WXRzAioV-ZcXZ-S0z7aQCZWW1kcd6rTgHB8jNWrB2j6xi1Lf7jkb3v3x2gqgxty5mJdQQ5M3sxghK6R2TCv3mmvPEfoqQlPwUkb1kWnm5ycejVeQurFjFOkFutK6WUPTojL73WQoUsJdaoznQv4OQteixFkALwO0SMeCDGv8fhN5FTrQ6GmvgT6Vy8A_vbeEpqKzNKCZYnCLrCMjtUWxd0cH6Hex89HGp9v_bYDwvEN2J0gziPB6eTEMhoD0EW-tDHwiPuohmkfMvC-Q1VvmToSFrfJjkUu30AxrH4mZlhnU0qR94dzJOOKWoOYE8Bg-u_rYlloEYzw0j6vOa7AYaBwf5_lWVQVMCu7XHi7aiiwlLK-8URWOfESHulE; 1P_JAR=2021-06-18-13; OTZ=6028663_44_44_123780_40_436260; SIDCC=AJi4QfEVOLRDrUJ8UBuVhsSMTMQuZnWnSUnHGBJ0TJH3wCzAP-3FtULbavNrgkaqRwoq6NEHnOk; __Secure-3PSIDCC=AJi4QfHCpeWgUnGHCA42qadTsrOFdx65BN6LVmrzf3kd8oI5_z5lOsZBIV7UtdywkJpZS1nUnkg' \
  --compressed \
  -o google_podcasts.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  134k    0  134k    0     0   177k      0 --:--:-- --:--:-- --:--:--  177k


In [86]:
with open("google_podcasts.html", "r") as infile:
    gp_html = infile.read()

soup = BeautifulSoup(gp_html)
parent = soup.find("div", {"class": "z9w95e"})
for div in parent:
    category = str(div.find("div", {"class": "ldf2Je"}).attrs["aria-label"])
    if "Топ подкастов в категории" not in category:
        continue
    category = category[27:-1]
    subdivs = div.find_all("div", {"class": "eWeGpe"})
    pd_dict["aggregator"] += ["Google" for _ in range(len(subdivs))]
    pd_dict["category"] += [category for _ in range(len(subdivs))]
    pd_dict["subcategory"] += [None for _ in range(len(subdivs))]
    pd_dict["score"] += [None for _ in range(len(subdivs))]
    pd_dict["title"] += [str(d.text) for d in subdivs]

## Get Listen Notes

In [ ]:
!pip install -q podcast-api

In [140]:
from listennotes import podcast_api
from credentials import listennotes_key

client = podcast_api.Client(api_key=listennotes_key)

In [116]:
genre_ids = {
    "99": "Новости",
    "82": "Досуг",
    "134": "Музыка",
    "69": "Религия и мистика",
    "107": "Наука",
    "122": "Общество и культура",
    "77": "Спорт",
    "68": "ТВ и кино",
    "127": "Технологии",
    "100": "Искусство",
    "93": "Бизнес",
    "111": "Образование",
    "168": "Нарратив",
    "88": "Здоровье и лайфстайл",
    "125": "История",
    "132": "Для всей семьи",
    "133": "Юмор"
}

In [119]:
for genre_id in genre_ids:
    for page in range(1, 11):
        resp = client.fetch_best_podcasts(
          genre_id=genre_id,
          page=str(page),
          region='ru',
          publisher_region='ru',
          safe_mode=0,
        )

        rjson = resp.json()
        megadump[genre_id] += [copy(rjson)]
        
        podcasts = rjson["podcasts"]
        pd_dict["aggregator"] += ["Listen Notes" for _ in range(len(podcasts))]
        pd_dict["category"] += [genre_ids[genre_id] for _ in range(len(podcasts))]
        pd_dict["subcategory"] += [None for _ in range(len(podcasts))]
        pd_dict["score"] += [p["listennotes_url"] for p in podcasts]
        pd_dict["title"] += [p["title"].strip() for p in podcasts]
        
        if not rjson['has_next']:
            break

with open("megadump.pkl", "wb") as outp:
    pickle.dump(megadump, outp)

In [130]:
podcasts_df = pd.DataFrame(pd_dict)

podcasts_df.columns = list(podcasts_df.columns)[:-1] + ["url"]
for colname in list(podcasts_df.columns):
    podcasts_df[colname] = [t.strip() if type(t) == str else t for t in list(podcasts_df[colname])]

podcasts_df

,title,aggregator,category,subcategory,url
0,Интересно до жути!,Top Yandex,Для всей семьи,Интересно сейчас,None
1,Все аудиошоу Детского радио!,Top Yandex,Для всей семьи,Интересно сейчас,None
2,Школьники расспрашивают иностранцев,Top Yandex,Для всей семьи,Интересно сейчас,None
3,Три отца пытаются воспитывать детей,Top Yandex,Для всей семьи,Интересно сейчас,None
4,В помощь родителям,Top Yandex,Для всей семьи,Редакция собирает для вас,None
...,...,...,...,...,...
3910,Культурные люди,Listen Notes,Юмор,None,https://www.listennotes.com/c/598484a282fe4796...
3911,Вечерний кефир,Listen Notes,Юмор,None,https://www.listennotes.com/c/f4c125d059fc40b7...
3912,Женщина. Руководство по эксплуатации,Listen Notes,Юмор,None,https://www.listennotes.com/c/1ff6302369a64a10...
3913,Анекдоты,Listen Notes,Юмор,None,https://www.listennotes.com/c/3601b5bee7b94d44...


In [88]:
podcasts_df.to_pickle("podcs.pickle")

## Analyze podcasts

In [131]:
yp_df = podcasts_df.loc[podcasts_df["aggregator"] == "Top Yandex"].reset_index(drop=True)
gp_df = podcasts_df.loc[podcasts_df["aggregator"] == "Google"].reset_index(drop=True)
ln_df = podcasts_df.loc[podcasts_df["aggregator"] == "Listen Notes"].reset_index(drop=True)

In [134]:
ln_df[ln_df['title'].isin(list(yp_df["title"]))].to_excel("top_podcasts.xlsx")

**(Manual analysis here)**

In [150]:
jingles_df = pd.read_excel("jingles_podcasts.xlsx")
jingles_df.columns = ["j_status", "title", "category_ln", "ln_id"]
jingles_df["ln_id"] = [u[30:-1] for u in list(jingles_df["ln_id"])]
jingles_df["category_yp"] = [list(yp_df["category"])[list(yp_df["title"]).index(str(t))] for t in list(jingles_df["title"])]
jingles_df = jingles_df[["j_status", "title", "category_ln", "category_yp", "ln_id"]]
jingles_df.to_pickle("jingles_df.pkl")
jingles_df

,j_status,title,category_ln,category_yp,ln_id
0,NaN,Детектив,Новости,Новости,f367abe58b8447b48853809bbd58d58e
1,NaN,Радио ТОП менеджмент FM | TMFM,Новости,Новости,12e3883765df499fb8fc504eb454663c
2,+,Задвижки по недвижке,Новости,Новости,a37edc43456b4fa5ae5c7839040dc01e
3,NaN,Байки из Отеля | Подкаст Эхо Москвы,Новости,Общество и культура,6652da4a9e27444590013eb17f6ebc57
4,NaN,Антонимы,Новости,Новости,0e78efe899064ec1a3b2a2a8350952f4
...,...,...,...,...,...
414,NaN,Сомнолог: налогово-медитативный подкаст,Юмор,Здоровье и лайфстайл,a2bdb477853740c387e579b2a4b15066
415,+,Хочу не могу 18+,Юмор,Общество и культура,3be59b3b87ec46b9a0979d8b2c0f0075
416,+,ERROR 404 | Некультурно о Культуре,Юмор,Юмор,26b10e05ae58415d8f3507c36138ec02
417,NaN,Мешок из под картошки,Юмор,Юмор,73e57fa482b045a0a550dcfb692b9b59
